# This project aims to build a computer vision deep learning model using convolutional neural network with open source data downloaded from Kaggle.

In [13]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [15]:
# defining our constants
IMAGE_SIZE = 256
BATCH_SIZE = 32  
Epochs = 50
CHANNELS = 3
dataset = tf.keras.preprocessing.image_dataset_from_directory( # importing our image data into tensorflow
    "PlantVillage",
    batch_size= BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,   # shuffling our data so to prevent bias
)

In [21]:
dataset_name = dataset.class_names
dataset_name

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

### 80/10/10 Train-Validation-Test Split

In [23]:
def train_val_test_split(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size)
    train_ds = ds.take(int(len(dataset)*train_split))
    val_ds = ds.skip(int(len(dataset)*train_split)).take(int(len(ds)*val_split))
    test_ds = ds.skip(int(len(dataset)*train_split)).skip(int(len(ds)*val_split))
    
    return train_ds, val_ds, test_ds

In [24]:
train_ds, val_ds, test_ds = train_val_test_split(dataset)
len(train_ds), len(val_ds), len(test_ds)

(54, 6, 8)

In [25]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

### Data Augmentation to add more data to prevent overfitting

In [26]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [27]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [34]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # 6 rounds of convolution and max pooling layers before dense layer
    # trials and error 
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (256, 256, 256)           0         
                                                                 
 conv2d_36 (Conv2D)          (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d_36 (MaxPooli  (32, 127, 127, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_37 (Conv2D)          (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_37 (MaxPooli  (32, 62, 62, 64)          0         
 ng2D)                                                           
                                                                 
 conv2d_38 (Conv2D)          (32, 60, 60, 64)        

In [35]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [36]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
)

Epoch 1/50
54/54 [==============================] - 117s 2s/step - loss: 0.8617 - accuracy: 0.5053 - val_loss: 0.7217 - val_accuracy: 0.5938
Epoch 2/50
54/54 [==============================] - 79s 1s/step - loss: 0.6342 - accuracy: 0.6937 - val_loss: 0.5022 - val_accuracy: 0.7552
Epoch 3/50
54/54 [==============================] - 77s 1s/step - loss: 0.3873 - accuracy: 0.8363 - val_loss: 0.2272 - val_accuracy: 0.9115
Epoch 4/50
54/54 [==============================] - 77s 1s/step - loss: 0.2967 - accuracy: 0.8744 - val_loss: 0.1341 - val_accuracy: 0.9583
Epoch 5/50
54/54 [==============================] - 77s 1s/step - loss: 0.1921 - accuracy: 0.9302 - val_loss: 0.1228 - val_accuracy: 0.9375
Epoch 6/50
54/54 [==============================] - 77s 1s/step - loss: 0.1429 - accuracy: 0.9396 - val_loss: 0.0760 - val_accuracy: 0.9740
Epoch 7/50
54/54 [==============================] - 78s 1s/step - loss: 0.0940 - accuracy: 0.9665 - val_loss: 0.0975 - val_accuracy: 0.9531
Epoch 8/50
54/54 [=

Validation loss is around 1% and validation accuracy is more than 99%. Our model is quite accurate at classifying novel potatoes disease images.

In [42]:
scores = model.evaluate(test_ds)

8/8 [==============================] - 2s 199ms/step - loss: 0.0090 - accuracy: 0.9961


In [43]:
scores

[0.009006748907268047, 0.99609375]

In [44]:
import os
model_version=max([int(i) for i in os.listdir("../models") + [0]])+1
model.save(f"../models/{model_version}")

INFO:tensorflow:Assets written to: ../models/3\assets


INFO:tensorflow:Assets written to: ../models/3\assets
